
# Stage 13 — Productization (Filled Notebook)

This notebook is **self-contained** and writes files into a local folder named `stage13_repo/` (relative to the notebook).
Run cells top-to-bottom to create:
- `src/model_utils.py` (modular code)
- `model/model.pkl` (pickled model)
- `app.py` (Flask API: POST/GET endpoints + `/plot`)
- `requirements.txt`, `README.md`, `reports/stakeholder_summary.md`

Then test the API using `curl` or the optional Python `requests` snippet below.


In [2]:

# 1) Create repo structure + modular utils
from pathlib import Path
import os
REPO = Path("./")

# REPO = Path("stage13_repo")
for sub in ["src","model","notebooks","reports","data"]:
    (REPO/sub).mkdir(parents=True, exist_ok=True)

(SRC := REPO/"src").joinpath("model_utils.py").write_text(
    "import numpy as np\n"
    "from sklearn.linear_model import LinearRegression\n\n"
    "def train_model(X, y):\n"
    "    model = LinearRegression()\n"
    "    model.fit(X, y)\n"
    "    return model\n\n"
    "def predict_model(model, X):\n"
    "    return model.predict(X)\n",
    encoding="utf-8"
)
print("Wrote", REPO/"src/model_utils.py")


Wrote src/model_utils.py


In [3]:

# 2) Train and pickle a tiny demo model
import numpy as np, pickle
from sklearn.linear_model import LinearRegression

X = np.array([[1],[2],[3],[4],[5]])
y = np.array([2,4,6,8,10])
model = LinearRegression().fit(X,y)

with open(REPO/"model/model.pkl","wb") as f:
    pickle.dump(model, f)
print("Saved model to stage13_repo/model/model.pkl")


Saved model to stage13_repo/model/model.pkl


In [4]:

# 3) Reload the pickle and test prediction
import pickle
with open(REPO / "model/model.pkl","rb") as f:
    reloaded = pickle.load(f)
print("Prediction for [[6]]:", reloaded.predict([[6]]).tolist())


Prediction for [[6]]: [12.000000000000002]


In [5]:

# 4) Write Flask API (app.py)
from pathlib import Path
(Path("stage13_repo")/"app.py").write_text(
    "from flask import Flask, request, jsonify, send_file\n"
    "import pickle, numpy as np, matplotlib.pyplot as plt\n"
    "from pathlib import Path\n\n"
    "app = Flask(__name__)\n\n"
    "with open('model/model.pkl','rb') as f:\n"
    "    model = pickle.load(f)\n\n"
    "@app.route('/predict', methods=['POST'])\n"
    "def predict_post():\n"
    "    data = request.get_json()\n"
    "    if not data or 'features' not in data:\n"
    "        return jsonify({'error':'Missing \'features\' key'}), 400\n"
    "    try:\n"
    "        X = np.array(data['features'])\n"
    "        preds = model.predict(X).tolist()\n"
    "        return jsonify({'predictions': preds})\n"
    "    except Exception as e:\n"
    "        return jsonify({'error': str(e)}), 400\n\n"
    "@app.route('/predict/<float:x>', methods=['GET'])\n"
    "def predict_get1(x):\n"
    "    try:\n"
    "        pred = model.predict([[x]]).tolist()\n"
    "        return jsonify({'prediction': pred})\n"
    "    except Exception as e:\n"
    "        return jsonify({'error': str(e)}), 400\n\n"
    "@app.route('/predict/<float:x>/<float:y>', methods=['GET'])\n"
    "def predict_get2(x, y):\n"
    "    try:\n"
    "        pred = model.predict([[x+y]]).tolist()\n"
    "        return jsonify({'prediction': pred})\n"
    "    except Exception as e:\n"
    "        return jsonify({'error': str(e)}), 400\n\n"
    "@app.route('/plot', methods=['GET'])\n"
    "def plot():\n"
    "    xs = np.linspace(0, 10, 50).reshape(-1,1)\n"
    "    ys = model.predict(xs)\n"
    "    plt.figure()\n"
    "    plt.plot(xs, ys, label='Prediction')\n"
    "    plt.legend(); plt.xlabel('x'); plt.ylabel('y')\n"
    "    img_path = Path('reports/plot.png')\n"
    "    img_path.parent.mkdir(parents=True, exist_ok=True)\n"
    "    plt.savefig(img_path)\n"
    "    return send_file(img_path, mimetype='image/png')\n\n"
    "if __name__ == '__main__':\n"
    "    app.run(debug=True)\n",
    encoding="utf-8"
)
print("Wrote", REPO / "app.py")


Wrote app.py


In [6]:

# 5) requirements.txt + README.md
from pathlib import Path
Path(REPO / "requirements.txt").write_text(
    "flask\nscikit-learn\nmatplotlib\nnumpy\npandas\nrequests\n",
    encoding="utf-8"
)
Path(REPO / "README.md").write_text(
    "# Stage 13 — Productization Demo\n\n"
    "## How to run\n"
    "1) pip install -r requirements.txt\n"
    "2) python app.py  # starts API at http://127.0.0.1:5000\n\n"
    "## Endpoints\n"
    "- POST /predict  (JSON: {\"features\": [[value]]})\n"
    "- GET  /predict/<x>\n"
    "- GET  /predict/<x>/<y>\n"
    "- GET  /plot  (returns PNG)\n",
    encoding="utf-8"
)
print("Wrote requirements.txt and README.md")


Wrote requirements.txt and README.md


In [7]:

# 6) Stakeholder summary
from pathlib import Path
Path('stage13_repo/reports/stakeholder_summary.md').write_text(
    "# Stakeholder Summary\n\n"
    "**Objective:** Minimal, handoff-ready productization pipeline.\n\n"
    "**Result:** Model trained, pickled, served via Flask API (with simple validation) and a plot endpoint.\n\n"
    "**Assumptions:** Linear baseline suffices for demo.\n\n"
    "**Risks:** No auth; minimal error handling.\n\n"
    "**Next Steps:** Add tests, logging, auth; containerize; CI/CD.\n",
    encoding="utf-8"
)
print("Wrote reports/stakeholder_summary.md")


Wrote reports/stakeholder_summary.md



## 7) API testing

Open a terminal in the `stage13_repo/` folder and run:
```bash
pip install -r requirements.txt
python app.py
```

Then in another terminal:
```bash
curl -s -X POST http://127.0.0.1:5000/predict   -H "Content-Type: application/json"   -d '{ "features": [[6]] }'

curl -s http://127.0.0.1:5000/predict/7
curl -s http://127.0.0.1:5000/predict/3/4
curl -s -o plot.png http://127.0.0.1:5000/plot
```
(Optional) Python `requests` smoke test (will fail gracefully if API isn't running).


In [8]:

# Optional: Python requests smoke test
import requests
for method, url, payload in [
    ("POST", "http://127.0.0.1:5000/predict", {"features": [[6]]}),
    ("GET",  "http://127.0.0.1:5000/predict/7", None),
]:
    try:
        if method == "POST":
            r = requests.post(url, json=payload, timeout=2)
        else:
            r = requests.get(url, timeout=2)
        print(method, url, "->", r.status_code, r.text[:120], "...")
    except Exception as e:
        print(method, url, "-> API not reachable (which is fine for now):", e)


POST http://127.0.0.1:5000/predict -> 200 {
  "predictions": [
    12.000000000000002
  ]
}
 ...
GET http://127.0.0.1:5000/predict/7 -> 404 <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the ...



## 8) Deliverables checklist (Stage 13)
- Repo folder: `stage13_repo/` (created locally by this notebook)  
- Modular code: `src/model_utils.py`  
- Model persisted: `model/model.pkl` (+ reload demo)  
- API file: `app.py` (POST/GET/plot)  
- Reproducibility: `requirements.txt`  
- Stakeholder summary: `reports/stakeholder_summary.md`  
- README with instructions: `README.md`  

> Submit this **notebook** plus the `stage13_repo/` folder (zipped) per your course instructions.
